In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
    Trainer
    ,TrainingArguments
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
import os 
from datasets import load_dataset
import pandas as pd

os.environ['HUGGING_FACE_HUB_TOKEN'] = os.getenv("HUGGING_FACE_HUB_TOKEN")

/home/ltnga/ITDSIU21079/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "DrissDo/Qwen2.5-3B-JohnMa"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_path = "DrissDo/Qwen2.5-3B-JohnMa"
tokenizer = AutoTokenizer.from_pretrained(model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading adapter weights from DrissDo/Qwen2.5-3B-JohnMa led to missing keys in the model: model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.self_attn.q_proj.lora_A.default.weight, model.layers.1.self_attn.q_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora_B.default.weight, model.layers.2.self_attn.q_proj.lora_A.default.weight, model.layers.2.self_attn.q_proj.lora_B.default.weight, model.layers.2.self_attn.v_proj.lora_A.default.weight, model.layers.2.self_attn.v_proj.lora_B.default.weight, model.laye

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048, padding_idx=151665)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.01, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): lora.Linear4bit(
            (base_layer): Linea

In [4]:
import json
import time
ids = []
questions = []
choices_A = []
choices_B = []
choices_C = []
choices_D = []
choices_E = []
answers=[]
with open('/home/ltnga/ITDSIU21079/VMLU/vmlu_v1.5/valid.jsonl', "r", encoding="utf-8") as f:
            lines = f.readlines()
            for line in lines:
                data = json.loads(line)
                ids.append(data["id"])
                questions.append(data["question"])
                answers.append(data["answer"])
                choices = data["choices"]
                try:
                    choices_A.append(choices[0])
                except:
                    choices_A.append('')
                try:
                    choices_B.append(choices[1])
                except:
                    choices_B.append('')
                try:
                    choices_C.append(choices[2])
                except:
                    choices_C.append('')
                try:
                    choices_D.append(choices[3])
                except:
                    choices_D.append('')
                try:
                    choices_E.append(choices[4])
                except:
                    choices_E.append('')

In [5]:
questions[1]

'Những yếu tố nào sau đây có thể dẫn đến thâm hụt cán cân thương mại của một nước:'

In [6]:
df = pd.DataFrame({
        "id": ids,
        "prompt": questions,
        "A": choices_A,
        "B": choices_B,
        "C": choices_C,
        "D": choices_D,
        "anwser":answers
    })

In [7]:
df["anwser"].value_counts()

anwser
C    202
D    191
A    181
B    170
Name: count, dtype: int64

In [8]:
from string import Template
preamble = \
        'Chỉ đưa ra chữ cái đứng trước câu trả lời đúng (A, B, C, D) của câu hỏi trắc nghiệm sau: '

template = Template('$preamble\n\n$prompt\n\n $a\n $b\n $c\n $d \nĐáp án:')

def format_input(df, idx):
    prompt = df.loc[idx, 'prompt']
    a = df.loc[idx, 'A']
    b = df.loc[idx, 'B']
    c = df.loc[idx, 'C']
    d = df.loc[idx, 'D']
    # e = df.loc[idx, 'E']

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)
        
    return input_text

In [9]:
from tqdm import tqdm
start = time.time()
for idx in df.index:
    
    inputs = tokenizer(format_input(df, idx), return_tensors="pt", return_token_type_ids=False).to(device)
    outputs = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1)
    
        # inputs = tokenizer(format_input(df, idx), return_tensors="pt").to(device)
        # outputs = model.generate(inputs["input_ids"], max_new_tokens=1)
    
        # inputs = tokenizer(format_input(df, idx), return_tensors="pt").to(device)
        # outputs = model.generate(**inputs, max_new_tokens=1)
    answer_decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    last_element = answer_decoded[-1]
    answer = last_element.split()[-1]
    answers.append(answer)

end = time.time()
duration = end - start
print('Time taken for running inference: ', duration)
df["answer_bot"]=answer


    

Time taken for running inference:  70.54584527015686


In [10]:
df['is_correct'] = df['anwser'] == df['answer_bot']  # Tạo cột kiểm tra đúng/sai
correct_count = df['is_correct'].sum()               # Tổng số câu đúng
total_count = len(df)                                # Tổng số câu hỏi

# Tính % đúng
accuracy = (correct_count / total_count) * 100
print(f"Phần trăm đúng: {accuracy:.2f}%")

Phần trăm đúng: 27.15%


In [11]:
# Before training Phần trăm đúng: 22.85%
# After training Phần trăm đúng: 27.15%